# Code for Section 6.4.3 Multiple CVaR Levels Case Study
This example performs CVaR portfolio optimization for the levels 90%, 95%, and 97.5%, illustrating how multiple portfolio optimization methods can be combined using Exposure Stacking as mentioned in https://ssrn.com/abstract=4709317. It uses the same simulation and constraints as https://ssrn.com/abstract=4217884 and https://ssrn.com/abstract=4825945.

In [1]:
import numpy as np
import pandas as pd
import fortitudo.tech as ft
from cvxopt import matrix

In [2]:
# Load instrument info and pnl
pnl = ft.load_pnl()
prior_stats = ft.simulation_moments(pnl)
np.round(prior_stats, 3)

,Mean,Volatility,Skewness,Kurtosis
Gov & MBS,-0.007,0.032,0.096,3.023
Corp IG,-0.004,0.034,0.107,3.109
Corp HY,0.019,0.061,0.173,2.971
EM Debt,0.027,0.075,0.217,3.057
DM Equity,0.064,0.149,0.396,3.148
EM Equity,0.080,0.269,0.766,4.099
Private Equity,0.137,0.278,0.716,3.758
Infrastructure,0.059,0.108,0.311,3.193
Real Estate,0.043,0.081,0.234,3.092
Hedge Funds,0.048,0.072,0.204,3.050


In [3]:
# Compute covariance matrix for resampling
covariance_matrix = ft.covariance_matrix(pnl).values
means = prior_stats['Mean'].values

In [4]:
# Price some options
put_90 = ft.put_option(1, 0.9, 0.16, 0, 1)
put_95 = ft.put_option(1, 0.95, 0.155, 0, 1)
put_atmf = ft.put_option(1, 1, 0.15, 0, 1)
call_atmf = ft.call_option(1, 1, 0.15, 0, 1)
call_105 = ft.call_option(1, 1.05, 0.145, 0, 1)
call_110 = ft.call_option(1, 1.1, 0.14, 0, 1)

# Compute relative P&L
S, I = pnl.shape
zeros_vec = np.zeros(S)
dm_equity_price = 1 + pnl['DM Equity'].values

put_90_pnl = np.maximum(zeros_vec, 0.9 - dm_equity_price) - put_90
put_95_pnl = np.maximum(zeros_vec, 0.95 - dm_equity_price) - put_95
put_atmf_pnl = np.maximum(zeros_vec, 1 - dm_equity_price) - put_atmf
call_atmf_pnl = np.maximum(zeros_vec, dm_equity_price - 1) - call_atmf
call_105_pnl = np.maximum(zeros_vec, dm_equity_price - 1.05) - call_105
call_110_pnl = np.maximum(zeros_vec, dm_equity_price - 1.1) - call_110

In [5]:
# Add option simulations to P&L
pnl['Put 90'] = put_90_pnl
pnl['Put 95'] = put_95_pnl
pnl['Put ATMF'] = put_atmf_pnl
pnl['Call ATMF'] = call_atmf_pnl
pnl['Call 105'] = call_105_pnl
pnl['Call 110'] = call_110_pnl

prior_stats2 = ft.simulation_moments(pnl)
np.round(prior_stats2, 3)

,Mean,Volatility,Skewness,Kurtosis
Gov & MBS,-0.007,0.032,0.096,3.023
Corp IG,-0.004,0.034,0.107,3.109
Corp HY,0.019,0.061,0.173,2.971
EM Debt,0.027,0.075,0.217,3.057
DM Equity,0.064,0.149,0.396,3.148
EM Equity,0.080,0.269,0.766,4.099
Private Equity,0.137,0.278,0.716,3.758
Infrastructure,0.059,0.108,0.311,3.193
Real Estate,0.043,0.081,0.234,3.092
Hedge Funds,0.048,0.072,0.204,3.050


# Specify CVaR optimization object

In [6]:
# Copy constraints from https://ssrn.com/abstract=4217884.
v = np.hstack((np.ones(I), [put_90, put_95, put_atmf, call_atmf, call_105, call_110]))
G = np.vstack((np.eye(len(v)), -np.eye(len(v))))
options_bounds = 0.5 * np.ones(6)
h = np.hstack((0.25 * np.ones(I), options_bounds, np.zeros(I), options_bounds))

alphas = [0.9, 0.95, 0.975]
R = pnl.values
cvar_opt = ft.MeanCVaR(R, G, h, v=v, alpha=alphas[0])

# Resampled optimization

In [7]:
# Parameter uncertainty specification
B = 99  # Number of efficient frontiers (99 to align with 3-fold Exposure Stacking)
P = 5  # Number of portfolios used to span the efficient frontiers
pf_index = 2
N = 100  # Sample size for parameter estimation
np.random.seed(3)  # Fix results
return_sim = np.random.multivariate_normal(means, covariance_matrix, (N, B))

In [8]:
# Mean uncertainty for the cash instruments
p = np.ones((S, 1)) / S
frontier_mean90 = np.full((len(v), P, B), np.nan)
frontier_mean95 = np.full((len(v), P, B), np.nan)
frontier_mean975 = np.full((len(v), P, B), np.nan)
results_dict = {alphas[0]: frontier_mean90, alphas[1]: frontier_mean95, alphas[2]: frontier_mean975}
for b in range(B):
    means_uncertainty = np.mean(return_sim[:, b, :], axis=0)
    q = ft.entropy_pooling(p, A=R[:, :10].T, b=means_uncertainty[:, np.newaxis])
    means_run = q.T @ R
    cvar_opt._expected_return_row = -matrix(np.hstack((means_run, np.zeros((1, 2)))))
    for alpha in results_dict.keys():
        cvar_opt._c[-1] = 1 / (1 - alpha)
        results_dict[alpha][:, :, b] = cvar_opt.efficient_frontier(P)

In [9]:
# Partition results
results = np.full((len(v), P, 3 * B), np.nan)
i = 0
for b in range(B):
    for alpha in results_dict.keys():
        results[:, :, i] = results_dict[alpha][:, :, b]
        i += 1

# Exposure Stacking
Below 3-fold Exposure Stacking is implemented for the three CVaR levels individually and combined.

In [10]:
pf_index = 2
es_90 = ft.exposure_stacking(3, frontier_mean90[:, pf_index, :])
es_95 = ft.exposure_stacking(3, frontier_mean95[:, pf_index, :])
es_975 = ft.exposure_stacking(3, frontier_mean975[:, pf_index, :])
es_all = ft.exposure_stacking(3, results[:, pf_index, :])

In [11]:
# Table
exposures = np.round(np.vstack((es_90, es_95, es_975, es_all)) * 100, 2).T
exposures_df = pd.DataFrame(
    exposures,
    index=prior_stats2.index,
    columns=['90%', '95%', '97.5%', 'Combined'])
exposures_df

,90%,95%,97.5%,Combined
Gov & MBS,0.00,0.10,0.56,0.02
Corp IG,0.00,0.00,0.00,-0.00
Corp HY,0.00,-0.00,0.00,0.00
EM Debt,12.85,11.73,10.71,11.91
DM Equity,0.85,11.08,18.76,9.84
EM Equity,0.00,0.00,-0.00,0.00
Private Equity,11.56,7.76,5.41,8.25
Infrastructure,24.23,20.44,17.22,21.06
Real Estate,20.46,16.59,14.12,17.01
Hedge Funds,25.00,25.00,25.00,25.00


In [ ]:
# pcrm-book - Next generation investment analysis.
# Copyright (C) 2025 Anton Vorobets.

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.